
## Example of PyVtu ##


Import and prep

In [1]:
# imports
import os
import sys
from pathlib import Path
import numpy as np
rng = np.random.default_rng()

import vtu
from vtu import PyVtu

Grab a base vtu, for example, by running a new trisurf from a tape

In [2]:
! cd tests && /opt/workspace/msc_project/cluster-trisurf/src/trisurf --force-from-tape tape -c iterations=10 && cd -

[2023-09-14 20:06:36] TRISURF-NG v. 13597e6, compiled on: Jul 30 2023 19:26:39.
[2023-09-14 20:06:36] Programming done by: Samo Penic and Miha Fosnaric
[2023-09-14 20:06:36] Released under terms of GPLv3
[2023-09-14 20:06:36] Starting program...

[2023-09-14 20:06:36] ************************************************
[2023-09-14 20:06:36] **** Generating initial geometry from tape *****
[2023-09-14 20:06:36] ************************************************

[2023-09-14 20:06:36] Starting initial_distribution on vesicle with 12 shells!...
[2023-09-14 20:06:36] initial_distribution finished!
[2023-09-14 20:06:36] using debug curvature model 7 (use old energy): set to 15 to use the new energy method
[2023-09-14 20:06:36] set to 0 to use only the old method (faster), set to 1 to use sum-angle gaussian curvature (E+= kx2*c1*c2)
[2023-09-14 20:06:36] simulation seed 1694711196
[2023-09-14 20:06:36] Setting volume V0=1800.56526697718800278
[2023-09-14 20:06:36] Setting area A0=897.895138643722

In [3]:
base_vtu = "tests/timestep_000009.vtu"

In [4]:
new_vtu = PyVtu(base_vtu)

This is a good opertunity to see what's in the vtu

In [5]:
print(new_vtu.tape[:400])

####### Vesicle definitions ###########
# nshell is a number of divisions of dipyramid
nshell=12
# dmax is the max. bond length (in units l_min)
dmax=1.7
# dmin_interspecies in the min. dist. between different vertex species (in units l_min)
dmin_interspecies=1.2
# bending rigidity of the membrane (in units kT)
xk0=20
# max step size (in units l_min)
stepsize=0.15


#######  Cell definitions #####


In [6]:
new_vtu.known_shorthands

{'Koordinates_tock': 'pos',
 'connectivity': ('blist', 'tlist'),
 'bond_list': 'blist',
 'triangle_list': 'tlist',
 'adhesion_strength': 'ad_w',
 'direct_force': 'f0',
 'bonding_strength': 'w',
 'spontaneous_curvature': 'c0',
 'curvature': 'c',
 'bending_energy': 'e',
 'vertices_idx': 'indices',
 'bending_modulus': 'k',
 'spontaneous_deviator': 'd0',
 'second_bending_modulus': 'k2'}

In [7]:
new_vtu.type[:10],new_vtu.pos[:10],new_vtu.c0[:10],new_vtu.f0[:10],new_vtu.w[:10]

(array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4]),
 array([[ 4.61912909e-16, -1.08795203e-15,  7.57052801e+00],
        [ 4.61912909e-16,  1.02078097e+00,  6.93965068e+00],
        [ 9.70820393e-01,  3.15438667e-01,  6.93965068e+00],
        [ 6.00000000e-01, -8.25829152e-01,  6.93965068e+00],
        [-6.00000000e-01, -8.25829152e-01,  6.93965068e+00],
        [-9.70820393e-01,  3.15438667e-01,  6.93965068e+00],
        [ 4.61912909e-16,  2.04156194e+00,  6.30877335e+00],
        [ 9.70820393e-01,  1.33621964e+00,  6.30877335e+00],
        [ 1.94164079e+00,  6.30877335e-01,  6.30877335e+00],
        [ 1.57082039e+00, -5.10390485e-01,  6.30877335e+00]]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

We need to change a few things:
* the tape file
* the type of the vertices we want to be cmcs
* the values of the properties of the cmcs

Changing the tape

In [8]:
new_vtu.tape = new_vtu.tape.replace("bond_model=0","bond_model=2")

change 50 of the bare membrane vertices to CMCs

In [9]:
bare_type = 4 # adhesive
cmc_type = 1|2|4|8|32 # active&bonding&adhesive&anisotropic&vicsek
bare_type, cmc_type

(4, 47)

In [10]:
index_bare = new_vtu.indices[new_vtu.type == bare_type]
index_cmc = new_vtu.indices[new_vtu.type == cmc_type]

check how many cmcs are already there. convert 50 of the bare vertices to cmcs

In [11]:
len(index_bare),len(index_cmc)

(722, 0)

In [12]:
rng.shuffle(index_bare)
new_vtu.type[index_bare[:50]] = cmc_type

In [13]:
index_bare = new_vtu.indices[new_vtu.type == bare_type]
index_cmc = new_vtu.indices[new_vtu.type == cmc_type]
len(index_bare),len(index_cmc)

(672, 50)

Change the values of the cmcs to the new ones.

In [14]:
# change all cmcs
new_vtu.f0[index_cmc] = 1
# change all bonding cmcs to have w 1
new_vtu.w[index_cmc] = 1
# change all their curvature and deviator
new_vtu.c0[index_cmc] = 1
new_vtu.d0[index_cmc] = -1

We now need to update and save

In [15]:
new_vtu.update_all()
new_vtu.write_vtu("tests/new_start.vtu")

We can now run trisurf on the new vtu

In [ ]:
! cd tests && /opt/workspace/msc_project/cluster-trisurf/src/trisurf --restore new_start.vtu -c iterations=20 && cd -

And this should be done